In [1]:
import pandas as pd
import numpy as np

In [2]:
from dgutils.pandas import add_column, add_columns

/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from utils import arr2db, EvalMetric

Using TensorFlow backend.
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from . import _csparsetools
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:182: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:184: Runti

In [4]:
# df = pd.read_pickle('result/debug.2020_02_06_1.pkl')
df = pd.read_pickle('result/debug.2020_02_06_2.pkl')

In [6]:
# subset to TS1
df = df[df['data_partition'] == 'TS1']

In [7]:
df.head()

,data_partition,len,one_idx,pred_val,seq,seq_id
2,TS1,78,"[[0, 1, 2, 3, 3, 4, 5, 7, 7, 7, 10, 10, 11, 12...","[[0.0, 2.0861626e-07, 1.7881393e-07, 1.8179417...",GGGCCCUUAGCUUAGUCUGGUUAAAGCGAUCGGCUCAUAACCGAUU...,4v8n-1-AW
9,TS1,145,"[[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 1...","[[0.0, 2.026558e-06, 3.2484531e-06, 5.5134296e...",GGGUGUCCGCACUAAGUUCGGCAUCAAUAUGGUGACCUCCCGGGAG...,5m73-1-A
10,TS1,35,"[[0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 12], [3...","[[0.0, 5.066395e-07, 1.4901161e-07, 8.940697e-...",GAGCAAGAGCCAUUGCACUCCGGUUUGAUGACCUC,4c4w-1-D
12,TS1,122,"[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16, 1...","[[0.0, 3.4868717e-06, 4.798174e-06, 2.115965e-...",CACCCCCGUGCCCAUAGCACUGUGGAACCACCCCACCCCAUGCCGA...,2zjq-1-Y
19,TS1,81,"[[0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 8, 9, 10, 11, ...","[[0.0, 4.172325e-07, 2.9802322e-07, 6.854534e-...",GCCCGCAUGGUGAAAUCGGUAAACACAUCGCACUAAUGCGCCGCCU...,5ah5-1-D


In [11]:
def calculate_metric(one_idx, pred_idx, seq_len, e):
    target = np.zeros((seq_len, seq_len))
    pred = np.zeros((seq_len, seq_len))
    target[one_idx] = 1
    pred[pred_idx] = 1
    sensitivity = e.sensitivity(pred, target)
    ppv = e.ppv(pred, target)
    return sensitivity, ppv, e.f_measure(sensitivity, ppv)

In [12]:
df_metric = []

eval_module = EvalMetric(bypass_pairing_check=True)
# thresholds = [0.0001, 0.0005, 0.001, 0.005, 0.007, 0.01]
thresholds = [0.0001, 0.001, 0.002, 0.003, 0.01]


for _, row in df.iterrows():
    seq_id = row['seq_id']
    seq_len = row['len']
    pred_val = row['pred_val']
    one_idx = row['one_idx']
#     print(seq_id, seq_len)
    for t in thresholds:
        _pred = (pred_val > t).astype(np.int)
        assert np.all((_pred == 1) | (_pred == 0))
        pred_idx = np.where(_pred == 1)
#         print(t)
#         print(calculate_metric(one_idx, pred_idx, seq_len, eval_module))
        metric_s, metric_p, metric_f = calculate_metric(one_idx, pred_idx, seq_len, eval_module)
        df_metric.append({
            'seq_id': seq_id,
            'len': seq_len,
            'threshold': t,
            'sensitivity': metric_s,
            'ppv': metric_p,
            'f_measure': metric_f,
        })

df_metric = pd.DataFrame(df_metric)


/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.
utils.py:327: RuntimeWarning: invalid value encountered in double_scalars
  return (2 * sensitivity * ppv)/(sensitivity + ppv)
utils.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return float(np.sum(target[idx_predicted_base_pair])/np.sum(pred))


In [13]:
for threshold in thresholds:
    print(threshold)
    print(df_metric[df_metric['threshold'] == threshold][['f_measure', 'sensitivity', 'ppv']].describe())
    print('')

0.0001
       f_measure  sensitivity        ppv
count  67.000000    67.000000  67.000000
mean    0.338536     0.668212   0.258302
std     0.217691     0.146605   0.228819
min     0.044444     0.076923   0.031250
25%     0.188047     0.595491   0.111941
50%     0.252492     0.678571   0.161905
75%     0.378091     0.763932   0.272910
max     1.000000     1.000000   1.000000

0.001
       f_measure  sensitivity        ppv
count  66.000000    67.000000  67.000000
mean    0.540414     0.574375   0.513128
std     0.197109     0.194268   0.234144
min     0.166667     0.000000   0.000000
25%     0.422710     0.480792   0.357257
50%     0.511370     0.575758   0.486486
75%     0.673729     0.711688   0.653010
max     1.000000     1.000000   1.000000

0.002
       f_measure  sensitivity        ppv
count  66.000000    67.000000  67.000000
mean    0.569079     0.502614   0.659169
std     0.208150     0.217609   0.234258
min     0.126582     0.000000   0.000000
25%     0.412903     0.351247   0.53